In [ ]:
import os, sys
if '/notebook' in os.getcwd():
    os.chdir('..')
if '.' not in sys.path:
    sys.path.append('.')

In [ ]:
# import libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup

## Question-answering sites

In [ ]:
url = 'https://stackexchange.com/sites?view=list'
with urlopen(url) as page:
    soup = BeautifulSoup(page, 'html.parser')
divs = soup.findAll('div', attrs= {'class': 'lv-info'})    
links = [div.find('a').attrs['href'] for div in divs]    
domains = [link.replace('https://', '') for link in links]

## Online news papers

In [ ]:
newspaperindex_url = 'https://www.newspaperindex.com'

def newspaperindex_extract_links(url):
    with urlopen(url) as page:
        soup = BeautifulSoup(page, 'html.parser')
    link_list = soup.find('div', attrs= {'class': 'cList'})
    links = [a.attrs['href'] for a in link_list.findAll('a')]
    links = [link if link.startswith('http') 
             else newspaperindex_url + link
             for link in links]
    return links

In [ ]:
country_links = [country_link for country_link in newspaperindex_extract_links(newspaperindex_url)]

In [ ]:
from tqdm import tqdm
newspaper_links = [link for country_link in tqdm(country_links)
                   for link in newspaperindex_extract_links(country_link)]

In [ ]:
re.match(r'((?:[^./]+\.)+[^./]+)', 'khaama.com')

In [ ]:
import re
newspaper_links = [re.sub(r'^https?://(www\.)?', '', link) for link in newspaper_links]

In [ ]:
newspaper_domains = set(re.match(r'(?:[^./]+\.)+[^./]+', link).group()
                        for link in newspaper_links)

## Exploring unscraped URLs

In [ ]:
from db import connect_db
conn = connect_db()
c = conn.cursor()

In [ ]:
#c.execute(f'UPDATE {visited_pages_table} SET processed = 0')
#conn.commit()

In [ ]:
c.execute(f'SELECT url from {visited_pages_table} WHERE processed = 0')
urls = [row['url'] for row in c.fetchall()]

In [ ]:
len(urls)

In [ ]:
import re
matchers = (re.match(r'https?://(?:[\d\w]+\.)+com/', url) for url in urls)
prefixes = (m.group() for m in matchers if m)
from collections import Counter
Counter(prefixes).most_common(5)

In [ ]:
url = 'https://www.bbc.com/sport/football/44595251'
from newspaper import Article
article = Article(url)
article.download()
article.parse()
print(article.authors)
print(article.publish_date)
print(article.top_image)
print(article.movies)
article.nlp()
print(article.keywords)
print(article.summary)